## <font color='darkblue'>Prefce</font>
Here we are going to use a toy testing environment `GridWorld` to demonstrate the usage of this lab.

### <font color='darkgreen'>Importing Packages</font>
Firstly, let's import all the necessary packages:

In [1]:
from skyline import lab
from skyline.lab import gridworld_env
from skyline.lab import gridworld_utils
from skyline.lab.alg import monte_carlo

### <font color='darkgreen'>Make Lab Environment</font>
We can list supported environment as below:

In [2]:
lab.list_env()

Then We use function <font color='blue'>make</font> to obtain the desired environment. e.g.:

In [3]:
grid_env = lab.make(lab.Env.GridWorld)

In [4]:
# Check what our environment looks like:
grid_env.info()

In [5]:
# Show available actions
grid_env.available_actions()

In [6]:
# Get current state
grid_env.current_state

Let's take a action and check the state change:

In [7]:
# Take action 'Up'
grid_env.step('U')

# Check current state
grid_env.current_state

After taking action `U`, we expect the axis-i to move up from 2->1 and we can confirm it from the output state. Let's reset the environment by calling method <font color='blue'>reset</font> which will bring the state of environment back to intial state `GridState(i=2, j=0)`:

In [8]:
# Reset environment
grid_env.reset()

# Check current state
grid_env.current_state

## <font color='darkblue'>Experiments of RL algorithms</font>
Here we are going to test some well-known RL algorithms and demonstrate the usage of this lab:

<a id='monte_carlo_method'></a>
### <font color='darkgreen'>Monte Carlo Method</font>
<b><font size='3ptx'>In this method, we simply simulate many trajectories (<font color='darkbrown'>decision processes</font>), and calculate the average returns.</font></b> ([wiki page](https://en.wikiversity.org/wiki/Reinforcement_Learning#Monte_Carlo_policy_evaluation))

We implement this algorithm in `monte_carlo.py`. The code below will demonstrate the usage of it:

In [9]:
from skyline.lab.alg import monte_carlo

In [10]:
mc_alg = monte_carlo.MonteCarlo()

In [11]:
grid_env.info()

In [12]:
grid_env.random_action(gridworld_env.GridState(1, 0))

#### Training

In [13]:
%%time
# Training
mc_alg.fit(grid_env)

Let's check what value function we get:

In [14]:
gridworld_utils.print_values(mc_alg._state_2_value, grid_env)

Then let's print the learned policy:

In [15]:
gridworld_utils.print_policy(mc_alg._policy, grid_env)

#### Prior run

Finally, let's reset the environment and play the game:

In [16]:
# Play game util done
grid_env.reset()

print(f'Begin state={grid_env.current_state}')
step_count = 0
while not grid_env.is_done:
    result = mc_alg.play(grid_env)
    step_count += 1
    print(result)
    
print(f'Final reward={result.reward} with {step_count} step(s)')

In [17]:
# Show learned value function
# mc_alg._state_2_value

In [18]:
# Show learned Q table
# mc_alg._q

<a id='random_method'></a>
### <font color='darkgreen'>Random Method</font>
This method takes random action in the given environment. It is often used as a based line to evaluate other RL methods.

In [19]:
from skyline.lab.alg import random_rl

In [20]:
random_alg = random_rl.RandomRL()

#### Train
Random won't require any training and therefore below call should end in no time.

In [21]:
%%time
# Training
random_alg.fit(grid_env)

#### Prior run
Since this is a random process, each time you play the game will have difference result:

In [22]:
# Play game util done
grid_env.reset()

print(f'Begin state={grid_env.current_state}')
step_count = 0
while not grid_env.is_done:
    result = random_alg.play(grid_env)
    step_count += 1
    print(result)
print(f'Final reward={result.reward} with {step_count} step(s)')

From the result above, it is obviously that <a href='#monte_carlo_method'><b>Monte Carlo Method</b></a> can perform much better than <a href='#random_method'><b>Random Method</b></a>!

## <font color='darkblue'>Scoreboard</font>
Before we know how score board work, we need to understand <b><font color='blue'>RLExaminer</font></b> first.

### <font color='darkgreen'>RLExaminer</font>
Every environment can have more than one examiner to calculate the score of RL method. Each examiner may have its own aspect to evaluate the RL method (time, reward etc.). Let's check one used to calculate the average reward of grid environment:

In [23]:
examiner = gridworld_env.GridWorldExaminer()

Then, what's score of `Monte Carlo Method`:

In [24]:
examiner.score(mc_alg, grid_env)

`Monte Carlo Method` got score 0.4. Let's check another RL method `Random Method`:

In [25]:
examiner.score(random_alg, grid_env)

`Random Method` got score 0.5 which is less than `Monte Carlo Method`.

### <font color='darkgreen'>Score Board</font>
<b><font color='blue'>Scoreboard</font></b> literally calculate the scores of given RL methods according to the specific examiner and the rank those RL methods accordingly:

In [26]:
score_board = lab.Scoreboard()

In [27]:
sorted_scores  = score_board.rank(
    examiner=examiner, env=grid_env, rl_methods=[random_alg, mc_alg])

In [28]:
sorted_scores

## <font color='darkblue'>A Real World RL problem (BCST test case selection)</font>
Here we are going to use a real-world RL example to explain how this lab works. From environment BCST, we want to select test cases strategically so to form a test case execution sequence which will obtain the maximum accumulated reward.

From BCST environment:
* Each action is a selected test case.
* State is the sequence of last executed test case sequence.
* A reward equal to 1 means the execution resulted in a crash/ramdump or 0 means nothing was caught.

### <font color='darkgreen'>Explore the environment</font>

In [29]:
import matplotlib.pyplot as plt
from skyline.lab import bcst_tc_env

#### Initialize environment and show info of it

In [30]:
bcst_env = bcst_tc_env.BCSTEnvironment()
bcst_examiner = bcst_tc_env.BCSTRewardCountExaminer()

bcst_env.info()

#### Supported actions

In [31]:
bcst_env.available_actions()

#### Available states

In [32]:
bcst_env.available_states()[:10]

#### Take a few actions to execute for experience

In [33]:
bcst_env.step('rl_test_case1')

In [34]:
bcst_env.step('rl_test_case2')
bcst_env.step('rl_test_case3')
bcst_env.current_state

In [35]:
bcst_env.step('rl_test_case4')
bcst_env.step('rl_test_case5')
bcst_env.current_state

### <font color='darkgreen'>Random RL</font>
Let's check our baseline `Random Method`:

#### Train

In [36]:
%%time
# Training
random_alg.fit(bcst_env)

#### Prior run

In [37]:
bcst_env.reset()
random_alg.play(bcst_env)

#### Score

In [38]:
score, _ = bcst_examiner.score(random_alg, bcst_env, play_round=10)
print(f'Score={score:.02f}')

### <font color='darkgreen'>Monte Carlo Method</font>
Next let's check `Monte Carlo Method` with three cases:
* `ml_alg_r1000`: Play 1000 times in BCST environment.
* `ml_alg_r3000`: Play 3000 times in BCST environment.
* `ml_alg_r5000`: Play 5000 times in BCST environment.

In [39]:
mc_alg_r1000 = monte_carlo.MonteCarlo(name='mc_r1000', round_num=1000)
mc_alg_r3000 = monte_carlo.MonteCarlo(name='mc_r3000', round_num=3000)
mc_alg_r5000 = monte_carlo.MonteCarlo(name='mc_r5000', round_num=5000)

#### Train

In [40]:
%%time
# Training by playing 1000 times in BCST environment.
mc_alg_r1000.fit(bcst_env)

In [41]:
%%time
# Training by playing 3000 times in BCST environment.
mc_alg_r3000.fit(bcst_env)

In [42]:
%%time
# Training by playing 5000 times in BCST environment.
mc_alg_r5000.fit(bcst_env)

#### Score

In [43]:
score, r1000_data = bcst_examiner.score(mc_alg_r1000, bcst_env, play_round=15)
print(f'Monte Carlo Method ({mc_alg_r1000.name}) with average score={score:.02f}')

In [44]:
score, r3000_data = bcst_examiner.score(mc_alg_r3000, bcst_env, play_round=15)
print(f'Monte Carlo Method ({mc_alg_r3000.name}) with average score={score:.02f}')

In [45]:
score, r5000_data = bcst_examiner.score(mc_alg_r5000, bcst_env, play_round=15)
print(f'Monte Carlo Method ({mc_alg_r5000.name}) with average score={score:.02f}')

In [46]:
fig, ax = plt.subplots()
ax.set_title('Monte Carlo Method (play_round=15; BCST will execute 500 steps in each round)')
ax.boxplot([r1000_data, r3000_data, r5000_data], labels=['r1000', 'r3000', 'r5000'])
plt.ylabel('Accumulated reward')
plt.xlabel('RL method name')
plt.minorticks_on()
plt.tick_params(axis='x', which='minor', bottom=False)
plt.tick_params(axis='x', which='major', labelsize='small')
plt.show()

### <font color='darkgreen'>BCST Test case selector</font>
From BCST in G3, we implement our own RL method.

In [47]:
from skyline.lab.alg import bcst_epsilon_tcs_rl

In [48]:
bcst_rl_method_r1000 = bcst_epsilon_tcs_rl.EGreedyStrategy(name='tcs_r1000', round_num=1000)
bcst_rl_method_r2000 = bcst_epsilon_tcs_rl.EGreedyStrategy(name='tcs_r1000', round_num=2000)
bcst_rl_method_r3000 = bcst_epsilon_tcs_rl.EGreedyStrategy(name='tcs_r1000', round_num=3000)

#### Train

In [49]:
%%time
# Training by playing 1000 times in BCST environment.
bcst_rl_method_r1000.fit(bcst_env)

In [ ]:
%%time
# Training by playing 2000 times in BCST environment.
bcst_rl_method_r2000.fit(bcst_env)

In [ ]:
%%time
# Training by playing 3000 times in BCST environment.
bcst_rl_method_r3000.fit(bcst_env)

In [ ]:
level_one_knowledge = bcst_rl_method_r3000.qtable['3'].items()
sorted(level_one_knowledge, key=lambda t: -t[1].accumulated_reward)[:3]

In [ ]:
bcst_rl_method_r3000._run_policy(bcst_env)

#### Score

In [ ]:
score, r1000_data = bcst_examiner.score(bcst_rl_method_r1000, bcst_env, play_round=30)
print(f'BCST TC selector ({bcst_rl_method_r1000.name}) with average score={score:.02f}')

In [ ]:
score, r2000_data = bcst_examiner.score(bcst_rl_method_r2000, bcst_env, play_round=30)
print(f'BCST TC selector ({bcst_rl_method_r2000.name}) with average score={score:.02f}')

In [ ]:
score, r3000_data = bcst_examiner.score(bcst_rl_method_r3000, bcst_env, play_round=15)
print(f'BCST TC selector ({bcst_rl_method_r3000.name}) with average score={score:.02f}')

In [ ]:
fig, ax = plt.subplots()
ax.set_title('BCST TC selector (play_round=30; BCST will execute 500 steps in each round)')
ax.boxplot([r1000_data, r2000_data, r3000_data], labels=['r1000', 'r3000', 'r5000'])
plt.ylabel('Accumulated reward')
plt.xlabel('RL method name')
plt.minorticks_on()
plt.tick_params(axis='x', which='minor', bottom=False)
plt.tick_params(axis='x', which='major', labelsize='small')
plt.show()

### <font color='darkgreen'>Score Board</font>
Finally, let's check the ranking among supported RL methods:

In [ ]:
sorted_scores  = score_board.rank(
    examiner=bcst_examiner, env=bcst_env,
    rl_methods=[random_alg, mc_alg_r1000, mc_alg_r3000, mc_alg_r5000,
                bcst_rl_method_r1000, bcst_rl_method_r2000, bcst_rl_method_r3000])

## <font color='darkblue'>Supplement</font>
* [Udemy - Artificial Intelligence: Reinforcement Learning in Python](https://www.udemy.com/course/artificial-intelligence-reinforcement-learning-in-python/)